In [117]:
#from Week_4 import G
import networkx as nx
G = nx.read_gpickle("css_graph.gpickle")

In [137]:
import netwulf as nw
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib.colors as mcolors
import math
import statistics
import random
from scipy.stats import ttest_1samp

In [138]:
#Make all missing top_fields = None
for author in G.nodes:
    if 'top_field' not in G.nodes[author]:
        G.nodes[author]['top_field'] = None


In [139]:
#Compute average neighboors in same top field as each node
def network_mean_same_field(top_fild_name):
    for author in G.nodes:
        G.nodes[author]['Avr_same_field'] = mean_same_field = statistics.mean(map(int, [G.nodes[author][top_fild_name] == field for field 
              in [G.nodes[neighboor][top_fild_name] for neighboor 
                  in G.neighbors(author)]]))
    
    #network average
    network_mean = statistics.mean(map(float, [G.nodes[author]['Avr_same_field'] for author in G.nodes]))
    return network_mean

network_mean = network_mean_same_field('top_field')

In [140]:
top_fields = list(nx.get_node_attributes(G, "top_field").values())

def randomise_top_fields_rand():
    random.shuffle(top_fields)
    for i, author in enumerate(G.nodes):
        G.nodes[author]['top_field_rand'] = top_fields[i]

In [141]:
#Compute average neighboors in same top field as each node 100 times for random top fields
rand_means = [] 
for i in range(100):
    randomise_top_fields_rand()
    rand_means.append(network_mean_same_field('top_field_rand'))
    
#rand_means


In [142]:
#compute the p-value for the null-hypothesis that out network mean occured by random... The p-value is extreemly low
t_stat, p_value = ttest_1samp(rand_means, network_mean)
p_value